# Analysis of the extension of area sources for different models

In [ ]:
%matplotlib inline
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('/Users/mpagani/Projects/original/oq-hazardlib/')
sys.path.append('/Users/mpagani/Projects/original/oq-engine/')
from utils_model import read_model, get_area
from shapely.wkt import loads
from openquake.hazardlib.source.area import AreaSource
from openquake.hazardlib.const import TRT

In [ ]:
sms = {}

modeln = './../../data/xml/emca_as.xml'
sms['emca'] = read_model(modeln)

modeln = './../../data/xml/emme_as_2015.xml'
sms['emme_as'] = read_model(modeln)

modeln = './../../data/xml/EOS_SEAsia.xml'
sms['sea'] = read_model(modeln)

In [ ]:
def get_model_data(sm):
    extn = []
    extr = {}
    for src in sm:
        if isinstance(src, AreaSource) and src.tectonic_region_type == TRT.ACTIVE_SHALLOW_CRUST:
            area = get_area(loads(src.polygon.wkt))
            extn.append(area)
            if src.tectonic_region_type in extr:
                extr[src.tectonic_region_type].append(area)
            else:
                extr[src.tectonic_region_type] = [area]
    return extn

In [ ]:
bins = np.logspace(2, 6, num=20)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)

cols = ['red', 'green', 'blue', 'orange']
colr = ['white', 'white', 'white', 'white']
lwdt = [4, 3, 2, 1]

for idx, key in enumerate(sms.keys()):

    extn = get_model_data(sms[key])
    print ('Min and Max value of log(area): {0:.2f} {1:.2f}'.format(np.log10(min(extn)), np.log10(max(extn))))
    
    histo, bin_edges = np.histogram(extn, bins=bins)
    hist = histo / float(sum(histo))
    
    plt.bar(bin_edges[0:-1]+np.diff(bin_edges)/2, hist, width=np.diff(bin_edges),
            edgecolor=colr[idx], facecolor=cols[idx], 
            linewidth=lwdt[idx], alpha=0.5)

    #plt.hist(extn, bins=bins, label=key, edgecolor=colr[idx],  
    #         normed=True, facecolor=cols[idx], 
    #         linewidth=lwdt[idx], alpha=0.5) 

plt.xlabel(r'Area, [sqkm]')
plt.grid(which='both', linestyle='--')
plt.xscale('log')
plt.legend()